In [ ]:
import numpy as np
import pandas as pd

In [ ]:
T = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
col = ['Size', 'BM', 'OP', 'INV']

In [ ]:
for t in T:
    df = pd.read_csv('../data/step2/value/{}.csv'.format(t))
    for c in col:
        a_20 = df[c].quantile(0.2)
        a_40 = df[c].quantile(0.4)
        a_60 = df[c].quantile(0.6)
        a_80 = df[c].quantile(0.8)
        for i in range(df.shape[0]):
            if df.loc[i, c] < a_20:
                df.loc[i, c] = 'A'
            elif df.loc[i, c] < a_40:
                df.loc[i, c] = 'B'
            elif df.loc[i, c] < a_60:
                df.loc[i, c] = 'C'
            elif df.loc[i, c] < a_80:
                df.loc[i, c] = 'D'
            else:
                df.loc[i, c] = 'E'
    df.to_csv('../data/analyze55/label/{}.csv'.format(t))
    

In [ ]:
finance = pd.read_csv('../data/step1/finance.csv')
returns = pd.read_csv('../data/step1/returns.csv')
for df in [finance, returns]:
    df['Time'] = pd.to_datetime(df['Time'])

In [ ]:
stk_group = []
for t in T:
    df = pd.read_csv('../data/analyze55/label/{}.csv'.format(t))
    for label1 in ['A', 'B', 'C', 'D', 'E']:
        for item in ['BM', 'OP', 'INV']:
            for label2 in ['A', 'B', 'C', 'D', 'E']:
                stk = df[(df['Size'] == label1) & (df[item] == label2)]['Stkcd'].tolist()
                stk_group.append((str(t) + '_' + item + '_' + label1 + label2, stk))
stk_group = dict(stk_group)

In [ ]:
R = []
for x in stk_group:
    [year, item, label] = x.split('_')
    year = int(year)
    m = 7
    
    df = pd.DataFrame(index=None)
    for st in stk_group[x]:
        tmp = returns[returns['Stkcd'] == st]
        df = pd.concat([df, tmp])
    
    for y in [year - 2, year - 1, year]:
        while (y < year and m <= 12) or (y == year and m <= 7):
            r = df[(df['Time'].dt.year == y) & (df['Time'].dt.month == m)][['Stkcd', 'Msmvosd', 'Mretwd']]
            r['S*R'] = r['Msmvosd'] * r['Mretwd']
            ans = r['S*R'].sum() / r['Msmvosd'].sum()
            R.append((str(year) + '_' + str(y) + str(m) + '_' + item + '_' + label, ans))
            m += 1
        m = 1
        
R = pd.DataFrame(R, index=None)
R.columns = ['label', 'R']
R.to_csv('../data/analyze55/R.csv')

In [ ]:
d = []
R = pd.read_csv('../data/analyze55/R.csv', index_col=0)
Y = [2014, 2015, 2016]

for year in Y:
    print(year - 2, '~', year)
    for c in ['BM', 'OP', 'INV']:
        print(c)
        
        for label1 in ['A', 'B', 'C', 'D', 'E']:
            v1 = 0
            v2 = 0
            v3 = 0
            v4 = 0
            v5 = 0
            m = 7
            for y in [year - 2, year - 1, year]:
                while (y < year and m <= 12) or (y == year and m <= 7):
                    s = str(year) + '_' + str(y) + str(m) + '_' + c + '_'
                    v1 += R[R['label'] == s + 'A' + label1]['R'].values
                    v2 += R[R['label'] == s + 'B' + label1]['R'].values
                    v3 += R[R['label'] == s + 'C' + label1]['R'].values
                    v4 += R[R['label'] == s + 'D' + label1]['R'].values
                    v5 += R[R['label'] == s + 'E' + label1]['R'].values
                    m += 1
                m = 1

            v1 = '%.2f'%(v1 * 100 / 25) + '%'  
            v2 = '%.2f'%(v2 * 100 / 25) + '%'
            v3 = '%.2f'%(v3 * 100 / 25) + '%' 
            v4 = '%.2f'%(v4 * 100 / 25) + '%' 
            v5 = '%.2f'%(v5 * 100 / 25) + '%'
            print(v1, v2, v3, v4, v5) 
            d.append([v1, v2, v3, v4, v5])

d = pd.DataFrame(d)
d.to_csv('../data/analyze55/example.csv')